In [ ]:
import open3d as o3d
import numpy as np
import copy
import subprocess
import os
import glob

#Define functions

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 4#2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 10#5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    fitness = 0
    count = 0
    maxAttempts = 20
    while fitness < 0.9999 and fitness < 1 and count < maxAttempts:
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(True),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
        if result.fitness > fitness and result.fitness < 1:
          fitness = result.fitness
          best_result = result
        count += 1
    return best_result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

def deformable_registration(source, target):
# Note: Windows users must change paths to'target.txt', 'source.txt', and 'bcpd' 
    sourceArray = np.asarray(source.vertices,dtype=np.float32)
    targetArray = np.asarray(target.vertices,dtype=np.float32)
    targetPath = './target.txt'
    sourcePath = './source.txt'
    np.savetxt (targetPath, targetArray, delimiter=',')
    np.savetxt (sourcePath, sourceArray, delimiter=',')
    path = './bcpd'
    cmd = f'"{path}" -x "{targetPath}" -y "{sourcePath}" -l10 -b10 -g0.1 -K140 -J500 -c1e-6 -p -d7 -e0.3 -f0.3 -ux -DB,5000,0.08 -ux -N1'
    subprocess.run(cmd, shell = True, check = True,text=True, capture_output=True)
    deformed_array = np.loadtxt('output_y.interpolated.txt')
    for fl in glob.glob("output*.txt"):
        os.remove(fl)
    os.remove(targetPath)
    os.remove(sourcePath)
    deformed = o3d.geometry.TriangleMesh()
    deformed.vertices = o3d.utility.Vector3dVector(deformed_array)
    deformed.triangles = source.triangles
    return deformed



In [ ]:
# Load and meshes and obtain point clouds
mesh1 = o3d.io.read_triangle_mesh("./meshes/O35_M_humerus_source.stl")
center1 = mesh1.get_center()
mesh1.translate(-center1, relative=False)
pcd1 = o3d.geometry.PointCloud()
pcd1.points = mesh1.vertices
pcd1.colors = mesh1.vertex_colors
pcd1.normals = mesh1.vertex_normals
mesh2 = o3d.io.read_triangle_mesh("./meshes/U35_F_humerus_partial_target.stl")
center2 = mesh2.get_center()
mesh2.translate(-center2, relative=False)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = mesh2.vertices
pcd2.colors = mesh2.vertex_colors
pcd2.normals = mesh2.vertex_normals

#Calculate object size
size = np.linalg.norm(np.asarray(pcd2.get_max_bound()) - np.asarray(pcd2.get_min_bound()))
voxel_size = size / 55

# Preprocessing (downsample, estimate normals, compute FPFH features)
source_down, source_fpfh = preprocess_point_cloud(pcd1, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(pcd2, voxel_size)

source_down.paint_uniform_color([1, 0.706, 0])
target_down.paint_uniform_color([0,0.706, 1])
o3d.visualization.draw_geometries([source_down,target_down])


In [ ]:
# Global registration
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

In [ ]:
# Refining the rigid registration
result_icp = refine_registration(pcd1, pcd2, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(pcd1, pcd2, result_icp.transformation)

In [ ]:
# Deformable registration
deformed = deformable_registration(mesh1.transform(result_icp.transformation),mesh2)
deformed.compute_vertex_normals()
mesh2.compute_vertex_normals()
deformed.paint_uniform_color([0,0.706, 0])
o3d.visualization.draw_geometries([deformed, mesh2])

In [ ]:
# Combine meshes (alternative - to crop the first before merging)
combined = deformed + mesh2
combined.compute_vertex_normals()
combined.paint_uniform_color([0,0.706, 0])
o3d.visualization.draw_geometries([combined])

In [ ]:
# Simplify mesh (smoothing and filtering)
mesh_smp = combined.simplify_vertex_clustering(voxel_size/3, contraction=o3d.geometry.SimplificationContraction.Average)
mesh_smp = mesh_smp.filter_smooth_simple(number_of_iterations=2)
mesh_smp = mesh_smp.filter_smooth_taubin(number_of_iterations=100)
mesh_smp.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_smp])